<a href="https://colab.research.google.com/github/gal1804/Click-Prediction/blob/main/Click_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install xgboost --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 255.9 MB 49 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
pip install us

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 21.9 MB/s 
  Created wheel for us: filename=us-2.0.2-py3-none-any.whl size=11945 sha256=c87142929a923a1964999ba8acd5e9a075492c1a2a07683d4489321af7227ed3
  Stored in directory: /root/.cache/pip/wheels/ca/6b/11/cda9ea2438f721330a35c9a2c8e34b4aedcd34c89af48a4d00
  Created wheel for jellyfish: filename=jellyfish-0.6.1-cp37-cp37m-linux_x86_64.whl size=72182 sha256=f6ec3d7d1990ae50a239ae28f74f144f4d57d1d34385a16fe4070a7135a4d96c
  Stored in directory: /root/.cache/pip/wheels/a1/99/51/7de469e37cd1b3c763c24394e1ebf1baa2d79e094bf346cf80
Successfully built us jellyfish


In [ ]:
import pandas as pd
import numpy as np
import datetime
from pandas import Timestamp
from datetime import datetime, timezone, timedelta
import us
from dateutil import tz
import xgboost as xgb
import sklearn as sk
from sklearn.model_selection import train_test_split
from google.colab import files
from sklearn.metrics import accuracy_score
import io
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/data/android_bids_us.csv"
df = pd.read_csv(path, header='infer')# Dataset is now stored in a Pandas Dataframe

In [ ]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))


def ex_time_of_day(hour_column): #peroid of day
    if (hour_column > 4) and (hour_column <= 8):
        return 'Early Morning'
    elif (hour_column > 8) and (hour_column <= 12 ):
        return 'Morning'
    elif (hour_column > 12) and (hour_column <= 16):
        return'Noon'
    elif (hour_column > 16) and (hour_column <= 20) :
        return 'Eve'
    elif (hour_column > 20) and (hour_column <= 24):
        return'Night'
    elif (hour_column <= 4):
        return'Late Night'
    
    
def ex_season(month_column): #season in year
    if (month_column >= 1) and (month_column <= 3):
        return 'winter'
    elif (month_column >= 4) and (month_column <= 6):
        return 'spring'
    elif (month_column >= 7) and (month_column <= 9):
        return'summer'
    elif (month_column >= 10) and (month_column <= 12) :
        return 'fall'
    
    
def convert(row):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(row['user_state_tz'])
    utc = row['utc_timestamp'].replace(tzinfo=from_zone)
    target = utc.astimezone(to_zone)
    return target.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]

In [ ]:
df['user_state_tz'] = df['user_state'].map(lambda x: us.states.lookup(x).capital_tz) #create a column with full state name
df['user_state_full_name'] = df['user_state'].map(lambda x: abbrev_to_us_state.get(x, x)) #create a column with a tz area from each state
df['utc_timestamp'] = pd.to_datetime(df['utc_time'], unit='ms') # epoch time to utc

df['local_time'] = df.apply(lambda row: convert(row), axis=1)
df['local_time'] = pd.to_datetime(df['local_time'], format='%Y-%m-%d %H:%M:%S.%f')
df['dayofweek'] = df['local_time'].dt.dayofweek #what day of the week this row belongs to

df['hour'] =  df['local_time'].dt.hour
df['month'] = df['local_time'].dt.month

df['timeofday'] = df['hour'].apply(ex_time_of_day)
#df['season'] = df['month'].apply(ex_season)


df = df.astype('category').replace('nan',np.nan)

In [ ]:
df = df.dropna(axis = 'rows', subset=['device_maker', 'device_model'])

df = df[['app_id', #leaving only relevant columns
         'user_state_full_name',
         'dayofweek',
         'user_isp',
         'device_maker',
         'device_model',
         'device_osv',
         'device_height',
         'device_width',
         'timeofday',
         #'season',
         'month',
         'click']]

In [ ]:
df.isnull().sum(axis = 0) #count Nans in every column

app_id                   0
user_state_full_name     0
dayofweek                0
user_isp                25
device_maker             0
device_model             0
device_osv               0
device_height            0
device_width             0
timeofday                0
month                    0
click                    0
dtype: int64

In [ ]:
df.dtypes

app_id                  category
user_state_full_name    category
dayofweek               category
user_isp                category
device_maker            category
device_model            category
device_osv              category
device_height           category
device_width            category
timeofday               category
month                   category
click                   category
dtype: object

In [ ]:
cols_lst = list(df.columns)
[[col, df[col].nunique()] for col in cols_lst] #num of unique values in every column

[['app_id', 13375],
 ['user_state_full_name', 51],
 ['dayofweek', 7],
 ['user_isp', 2540],
 ['device_maker', 265],
 ['device_model', 2342],
 ['device_osv', 16],
 ['device_height', 272],
 ['device_width', 218],
 ['timeofday', 6],
 ['month', 3],
 ['click', 2]]

In [ ]:
len(df)

2184800

In [ ]:
df.groupby(['timeofday', 'month', 'click']).size()

timeofday      month  click
Early Morning  8      0             0
                      1             0
               9      0        100879
                      1          7941
               10     0        111718
                      1          9467
               11     0         30099
                      1          2757
Eve            8      0             0
                      1             0
               9      0         80658
                      1          6243
               10     0        237119
                      1         18339
               11     0        109125
                      1          8191
Late Night     8      0             0
                      1             0
               9      0         84981
                      1          6159
               10     0        101506
                      1          7810
               11     0         29336
                      1          2427
Morning        8      0             0
                      

In [ ]:
# Supported tree methods are `gpu_hist`, `approx`, and `hist`.
clf = xgb.XGBClassifier(
    tree_method="gpu_hist", enable_categorical=True, use_label_encoder=False
)

NameError: ignored

In [ ]:
Y_col = 'click'
X_cols = df.loc[:, df.columns != Y_col].columns

X_train, X_test, y_train, y_test = train_test_split(df[X_cols], df[Y_col],test_size=0.2, random_state=14)

In [ ]:
# X is the dataframe we created in previous snippet
clf.fit(X_train, y_train)
# Must use JSON/UBJSON for serialization, otherwise the information is lost.
clf.save_model("categorical-model.json")

In [ ]:
y_pred = clf.predict(X_test)
predictions = [round(value) for value in y_pred]

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 90.57%


In [ ]:
y_train

607561     0
1615613    0
745062     0
2065607    0
2352132    0
          ..
2050799    0
641120     0
2146875    0
2253997    0
1305329    0
Name: click, Length: 2519062, dtype: category
Categories (2, int64): [0, 1]

In [ ]:
type(y_test)

pandas.core.series.Series

In [ ]:
df[Y_col]

0          0
1          0
2          0
3          0
4          0
          ..
3148823    0
3148824    0
3148825    0
3148826    0
3148827    0
Name: click, Length: 3148828, dtype: category
Categories (2, int64): [0, 1]

In [ ]:
df.head()

,app_id,user_state_full_name,dayofweek,user_isp,device_maker,device_model,device_osv,device_height,device_width,timeofday,month,click
46,com.critical.strike.shoot.fire.fssc,California,0,AT&T Wireless,Samsung,SM-G920AZ,7.0,1080,1920,Night,9,0
47,com.cardgame.solitaire,Alabama,1,AT&T Wireless,Samsung,SM-G955U,7.0,1280,720,Late Night,9,0
48,com.xiart.fb.towersx,California,0,Google Fiber,Samsung,SM-G930V,8.0.0,1080,1920,Night,9,1
49,com.thumbspire.weedfirm2,North Carolina,1,T-Mobile USA,LG,MS210,7.0,1280,720,Late Night,9,0
50,com.zentertain.classicvegasslots,Georgia,1,Sprint PCS,LG,LS777,7.0,720,1280,Late Night,9,0


In [ ]:
df.isnull().sum(axis = 0) #count Nans in every column

app_id                       0
user_state_full_name         0
dayofweek                    0
user_isp                    39
device_maker            964004
device_model            963858
device_osv                   0
device_height                0
device_width                 0
timeofday                    0
month                        0
click                        0
dtype: int64